# Парсер

#### Импортируем и настраиваем все

In [104]:
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
import chromedriver_binary 
import urllib
import time
from selenium.webdriver.common.keys import Keys
import lxml
import requests
import base64
import json
import pandas as pd
import numpy as np
import urllib.request
from fake_useragent import UserAgent
from bs4 import BeautifulSoup 
from selenium.webdriver.common.action_chains  import ActionChains

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

#####  Подготовка к работе с API, авторизируемся на сайте и изменяем кодировку для получения токена доступа.

In [105]:
url = "https://accounts.spotify.com/api/token"
headers = {}
data = {}

message = f"{'3e661d7345d04909b518ec04ec996fd2'}:{'c3ce130028654a779d6404a619845bd6'}"
messageBytes = message.encode('ascii')
base64Bytes = base64.b64encode(messageBytes)
base64Message = base64Bytes.decode('ascii')

headers['Authorization'] = f"Basic {base64Message}"
data['grant_type'] = "client_credentials"

r = requests.post(url, headers=headers, data=data)

token = r.json()['access_token']

In [106]:
token

'BQAsoyv_4LYEJCJ5Z5zI3S_UN0akV_zVojqgrBKv85MFiucmqajOXguSDwrTnRkwKhHx5GYd7_fC5LyDFtQ'

Успешно получен токен.

## Пишем функции для получения и обработки данных

###### Функция получения сведений о треках, находящихся в одном плейлисте (и ее мини-версия)

In [107]:
def get_playlist(playlist_id):
    playlistUrl = f'https://api.spotify.com/v1/playlists/{playlist_id}/tracks'
    headers = {
        "Authorization": "Bearer " + token
    }
    
    page = requests.get(url=playlistUrl, headers=headers)
    page =  page.json()
    page = playlist_req_to_list (page)    
    return page


In [108]:
def mini_get_playlist(playlist_id):
    playlistUrl = f'https://api.spotify.com/v1/playlists/{playlist_id}/tracks'
    headers = {
        "Authorization": "Bearer " + token
    }
    
    page = requests.get(url=playlistUrl, headers=headers)
    page =  page.json()
    page = mini_playlist_req_to_list(page)    
    return page

###### Функция получения параметров песни

In [109]:
def song_paramets(song_id):
    
    song_url = f'https://api.spotify.com/v1/audio-features/{song_id}'
    headers = {
        "Authorization": "Bearer " + token
    }

    page = requests.get(url=song_url, headers=headers)
    page =  page.json()
    return page

###### Функция для распредения инфомации о песнях из json-формата в DataFrame-таблицу (и ее мини-версия)

In [110]:
def playlist_req_to_list(file):

    song_name = [None for i in range(50)]
    song_id = [None for i in range(50)]
    duration = [None for i in range(50)]
    artist_name = [None for i in range(50)]
    artist_id = [None for i in range(50)]
    album_name = [None for i in range(50)]
    album_id = [None for i in range(50)]
    release_date = [None for i in range(50)]
    local_position = [None for i in range(50)]
    world_position = [None for i in range(50)]
    danceability = [None for i in range(50)]
    energy = [None for i in range(50)]
    key = [None for i in range(50)]
    loudness = [None for i in range(50)]
    mode = [None for i in range(50)]
    speechiness = [None for i in range(50)]
    acousticness = [None for i in range(50)]
    instrumentalness = [None for i in range(50)]
    liveness = [None for i in range(50)]
    valence = [None for i in range(50)]
    tempo = [None for i in range(50)]
      
    i = 0
    
    for item in file['items']:        
        song_name[i] = item['track']['name']
        song_id[i] = item['track']['id']
        duration[i] = item['track']['duration_ms']
        local_position[i] = item['track']['popularity']       
        artist_name[i] = item['track']['album']['artists'][0]['name']
        artist_id[i] = item['track']['album']['artists'][0]['id']
        album_name[i] = item['track']['album']['name']
        album_id[i] = item['track']['album']['id']
        release_date[i] = item['track']['album']['release_date']
        
        parametrs = song_paramets(song_id[i])
        
        danceability[i] = parametrs['danceability']
        energy[i] = parametrs['energy']
        key[i] = parametrs['key']
        loudness[i] = parametrs['loudness']
        mode[i] = parametrs['mode']
        speechiness[i] = parametrs['speechiness']
        acousticness[i] = parametrs['acousticness']
        instrumentalness[i] = parametrs['instrumentalness']
        liveness[i] = parametrs['liveness']
        valence[i] = parametrs['valence']
        tempo[i] = parametrs['tempo']
        
        i += 1
        
    data_dict = {'song_name' : song_name,
                 'song_id' : song_id,
                 'duration_ms' : duration,
                 'local_position' : local_position,
                 'artist_name' : artist_name,
                 'artist_id' : artist_id,
                 'album_name' : album_name,
                 'album_id' : album_id,
                 'release_date' : release_date,
                 'danceability': danceability,
                 'energy': energy,
                 'key': key,
                 'loudness': loudness,
                 'mode': mode,
                 'speechiness': speechiness,
                 'acousticness': acousticness,
                 'instrumentalness': instrumentalness,
                 'liveness': liveness,
                 'valence': valence,
                 'tempo': tempo
                }
    data_set = pd.DataFrame(data_dict)
    
    return data_set

In [111]:
def mini_playlist_req_to_list(file):

    song_name = [None for i in range(50)]
    song_id = [None for i in range(50)]
      
    i = 0
    
    for item in file['items']:        
        song_name[i] = item['track']['name']
        song_id[i] = item['track']['id']
        
        i += 1
        
    data_dict = {'song_name' : song_name, 'song_id' : song_id}
    data_set = pd.DataFrame(data_dict)
    
    return data_set

##### Функция по получению основной инфы о плейлисте

In [112]:
def playlist_info(playlist_id):
    playlistUrl = f'https://api.spotify.com/v1/playlists/{playlist_id}'
    headers = {
        "Authorization": "Bearer " + token
    }
    
    page = requests.get(url=playlistUrl, headers=headers)
    page =  page.json()
    text = page['description']
    number = page['followers']['total']
    country = text[text.find('now -')+6:-1]
    info = [playlist_id,country,number]
    return info

##### Функция по получению для каждого плейлиста списка песен, их места в топе, прослушиваний и т.д.

In [113]:
def get_MainInfo(playlist_id):
    #создаем часть таблицы (место в топе в качестве индекса, имена и id песен)
    MainInfo = mini_get_playlist(playlist_id)[['song_name', 'song_id']]
    
    #парсим код страницы альбома с испльзованием селениума
    site = "https://open.spotify.com/playlist/" + playlist_id
    chrome_options = Options()
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(site)
    time.sleep(3)
    response = driver.page_source
    driver.close()
    tree = BeautifulSoup(response, 'html.parser')
    
    #ищем все куски кода, где написаны просмотры
    a = tree.find_all('div', class_ = 'd47b790d001ed769adcd9ddfc0e83acc-scss f3fc214b257ae2f1d43d4c594a94497f-scss')
    
    #из них получаем строки с прослушиваниями и обрабатываем их
    res = []
    b = list(a)
    for i in b:
        views = str(i)[-16:-6].replace('\xa0', '')
        res.append(views)
    #res - список прослушиваний от топа ко дну (неформатированный)
    #вытащим из элементов res только числа
    views = []
    for i in res:
        ddd = [s for s in i.split('>') if s.isdigit()]
        views.append(int(ddd[0]))
    #views - просмотры по порядку для каждой песни из топа
    #добавляем колонки с просмотрами, местом в топе и страной к существующей таблице
    MainInfo.insert(2, "views", views)
    MainInfo.insert(0, "place", range(1,51))
    MainInfo.insert(0, "country", playlist_info(playlist_id)[1])
    return MainInfo

##### функция для получения жанров по исполнителю

In [114]:
def get_genres(artists):
    result = []
    for i in tqdm(artists):
        id = i
        playlistUrl = f'https://api.spotify.com/v1/artists/{id}'
        headers = {"Authorization": "Bearer " + token}
        page = requests.get(url=playlistUrl, headers=headers)
        page = page.json()
        genres = page['genres']
        result.append(genres)
        time.sleep(0.2)
    return result

## Находим все айдишники альбомов 
#### Полученное ids-список айдишников

In [115]:
site = "https://open.spotify.com/genre/charts-regional-weekly"

#chrome_options = Options() работает у Дениса
#chrome_options.add_argument("--headless")   настройка на всякий случай

#настраиваем селениум
driver = webdriver.Chrome('D:\chromedriver.exe') #работает у Дениса
#driver = webdriver.Chrome('/Users/alexandr/Downloads/chromedriver') #работает у Сани
#открываем сайт, двигаем мышку и кликаем, чтобы потом заработала прокрутка
driver.get(site)
driver.maximize_window()
time.sleep(3)
ActionChains(driver).move_by_offset(1520, 350).click().perform() #у дениса 1520 350
time.sleep(3)
driver.find_element_by_xpath('//body').send_keys(Keys.END) 
time.sleep(3)
driver.find_element_by_xpath('//body').send_keys(Keys.END) #моя добавка
time.sleep(3)


#получаем код страницы и делаем суп
response = driver.page_source
driver.close()
tree = BeautifulSoup(response, 'html.parser')

In [116]:
#ищем все ссылки на странице
a = tree.find_all('a', class_ = 'f7ebc3d96230ee12a84a9b0b4b81bb8f-scss')

#из блоков с ссылками достаем айдишники плейлистов
ids = []
for i in a:
    id = i.get('href')
    ids.append(id[10:])

# Cобираем датасеты

### Структура данных:
##### 1) Маленький датасет вида (айди листа, страна, кол-во подписчиков)
##### 2) Датасет побольше (данные о каждой песне, включая ее акустические характеристики)
##### 3) Датасет совсем большой че с ним (данные о позиции и прослушиваниях каждой песни в каждом топе)

### Маленький датасетик


In [118]:
countries = pd.DataFrame(playlist_info(item) for item in ids[:5])
countries.columns= ['id', 'country', 'followers'] 
countries = countries.set_index('country')


### Датасет побольше (данные о каждой песне)


In [119]:
allsongs=pd.DataFrame()
for item in tqdm(ids[:5]):
    allsongs=pd.concat([allsongs,get_playlist(item)],sort=False, axis=0, )

allsongs = allsongs.drop_duplicates(subset=['song_id'])
allsongs = allsongs.set_index('song_id')

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [01:22<00:00, 16.49s/it]


In [120]:
#добавим инфу о жанрах в allsongs
artists = list(allsongs['artist_id'])
genres = get_genres(artists)
allsongs['genres'] = genres

100%|████████████████████████████████████████████████████████████████████████████████| 147/147 [02:18<00:00,  1.06it/s]


### Датасет совсем большой че с ним (данные о позиции и прослушиваниях каждой песни в каждом топе)

In [121]:
positions = pd.DataFrame()
for id in tqdm(ids[:5]):
    country = get_MainInfo(id)
    positions = pd.concat([positions, country],sort=False, axis=0)
    
positions = positions.reset_index(drop = True)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [02:08<00:00, 25.62s/it]
